# Shared code block

In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = (256, 256)

def load_images_from_folder(folder, size=IMAGE_SIZE):
    imgs = []
    files = sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
    ])

    for fname in files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Skipping unreadable: {fname}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        img = img.astype("float32") / 255.0
        imgs.append(img)

    return np.array(imgs)


def build_autoencoder(input_shape=(256, 256, 3)):
    inp = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    x = MaxPooling2D((2,2), padding="same")(x)

    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2), padding="same")(x)

    # Bottleneck
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)

    # Decoder
    x = UpSampling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)

    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, (3,3), activation="relu", padding="same")(x)

    out = Conv2D(3, (3,3), activation="sigmoid", padding="same")(x)

    model = Model(inp, out)
    return model

def nightvis_weighted_mae(y_true, y_pred):
    # brightness mask
    lum = tf.reduce_mean(y_true, axis=-1, keepdims=True)
    weights = 1 + 4 * lum

    return tf.reduce_mean(weights * tf.abs(y_true - y_pred))


In [2]:
# blur model -> working, needs small tuning
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)
print(input_images.shape)

print("Loading BLUR style images...")
blur_images = load_images_from_folder(STYLE_FOLDER)
print(blur_images.shape)

if len(input_images) != len(blur_images):
    raise ValueError("Mismatch in BLUR training set!")

model = build_autoencoder()
model.fit(
    input_images,
    blur_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_blur.keras")
print("Saved BLUR model.")


Loading input images...
(13, 256, 256, 3)
Loading BLUR style images...
(13, 256, 256, 3)
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 0.2261 - val_loss: 0.2517
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 761ms/step - loss: 0.2150 - val_loss: 0.2553
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 854ms/step - loss: 0.2015 - val_loss: 0.2187
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 727ms/step - loss: 0.1911 - val_loss: 0.2369
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 717ms/step - loss: 0.1887 - val_loss: 0.2259
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 724ms/step - loss: 0.1721 - val_loss: 0.2183
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 733ms/step - loss: 0.1569 - val_loss: 0.2550
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 770ms/step - loss: 0.1536 - val_loss: 0.1998
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 743ms/step - loss: 0.1538 - val_loss: 0.1896
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 741ms/step - loss: 0.1291 - val_loss: 0.2294
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 713ms/step - loss: 0.1

In [4]:
# night_vis

import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/night_vis"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading Night Vision style images...")
night_vis_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(night_vis_images):
    raise ValueError("Mismatch in Night Vision training set!")

model = build_autoencoder()
model.compile(
    optimizer=Adam(1e-3),
    loss=nightvis_weighted_mae,
    metrics=["mae"]
)

model.fit(
    input_images,
    night_vis_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_night_vis.keras")
print("Saved  Night Vision model.")


Loading input images...
Loading Night Vision style images...
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 996ms/step - loss: 0.6343 - mae: 0.3992 - val_loss: 0.5290 - val_mae: 0.3473
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 755ms/step - loss: 0.5037 - mae: 0.3317 - val_loss: 0.3560 - val_mae: 0.2367
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 749ms/step - loss: 0.3647 - mae: 0.2392 - val_loss: 0.3035 - val_mae: 0.1570
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 822ms/step - loss: 0.3487 - mae: 0.1774 - val_loss: 0.3245 - val_mae: 0.1981
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 740ms/step - loss: 0.3244 - mae: 0.1896 - val_loss: 0.2919 - val_mae: 0.1517
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 731ms/step - loss: 0.3035 - mae: 0.1649 - val_loss: 0.2804 - val_mae: 0.1479
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 733ms/step - loss: 0.3007 - mae: 0.1546 - val_loss: 0.2992 - val_mae: 0.1755
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 899ms/step - loss: 0.2782 - mae: 0.1469 - val_loss: 0.2990 - val_mae: 0.1289
Epo

In [4]:

# outline
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/outline"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading OUTLINE style images...")
outline_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(outline_images):
    raise ValueError("Mismatch in OUTLINE training set!")

model = build_autoencoder()
model.fit(
    input_images,
    outline_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_outline.keras")
print("Saved OUTLINE model.")


Loading input images...
Loading OUTLINE style images...
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 0.2500 - val_loss: 0.2555
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 779ms/step - loss: 0.2424 - val_loss: 0.2477
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 742ms/step - loss: 0.2358 - val_loss: 0.2382
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 746ms/step - loss: 0.2261 - val_loss: 0.2277
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 837ms/step - loss: 0.2140 - val_loss: 0.2331
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 816ms/step - loss: 0.2120 - val_loss: 0.2175
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 973ms/step - loss: 0.2076 - val_loss: 0.2130
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 888ms/step - loss: 0.2031 - val_loss: 0.2043
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 818ms/step - loss: 0.1985 - val_loss: 0.2064
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 730ms/step - loss: 0.1969 - val_loss: 0.2003
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step - loss: 0.1919 - val_loss: 0.2012
Epoch 12/2

In [5]:
# poster style
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/poster"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading POSTER style images...")
poster_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(poster_images):
    raise ValueError("Mismatch in POSTER training set!")

model = build_autoencoder()
model.fit(
    input_images,
    poster_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_poster.keras")
print("Saved POSTER model.")


Loading input images...
Loading POSTER style images...
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 943ms/step - loss: 0.3369 - val_loss: 0.2618
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 822ms/step - loss: 0.3235 - val_loss: 0.2564
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 763ms/step - loss: 0.2970 - val_loss: 0.3131
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 747ms/step - loss: 0.2600 - val_loss: 0.3805
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 846ms/step - loss: 0.2647 - val_loss: 0.4226
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 761ms/step - loss: 0.2545 - val_loss: 0.4257
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 824ms/step - loss: 0.2515 - val_loss: 0.4205
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 770ms/step - loss: 0.2461 - val_loss: 0.3998
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 872ms/step - loss: 0.2326 - val_loss: 0.3557
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 864ms/step - loss: 0.2183 - val_loss: 0.3418
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 861ms/step - loss: 0.1950 - val_loss: 0.2965
Epoch 12